# Intro 

The goal of this project is in conjuction with Zyfra, a company whose goal is to develop efficiency solutions for heavy industry. This project specifically will be for gold mining. Prior to beginning this project, I had to become familiar with the process of gold extraction and the different stages and substances I would soon be dealing with. Once familiar, I'll begin by loading and preparing the data, making it ready for analysis. I will then find different metrics that give insight into the process of various stages and purifications. This will give insight into what areas of recovery may be more valuable than others. I'll finish by testing different models with the target of predicting the amount of gold recovered from gold ore. By doing this, I hope to help maximize production efforts while also eliminating unprofitable parameters. 

# Preparing the Data

In [ ]:
# importing libraries 
import pandas as pd 
from sklearn.metrics import mean_absolute_error 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [ ]:
train = pd.read_csv('/datasets/gold_recovery_train.csv')
train

In [ ]:
train.describe()

In [ ]:
test = pd.read_csv('/datasets/gold_recovery_test.csv')
test

In [ ]:
test.describe()

In [ ]:
full = pd.read_csv('/datasets/gold_recovery_full.csv')
full

In [ ]:
full.describe()

Upon loading the data and finding some general info, each dataframe seems to be well structured and without any issues. I will then proceed with some analysis, first by determining if recovery is calculated correctly.

I will now analyze the features that are not available in the test set.

In [ ]:
# analyzing against the training set as the full set is a combination of both and won't separate needed contexts
features_only_in_train = set(train.columns) - set(test.columns)
features_only_in_train 

In [ ]:
# finding the dtypes of the features above 
for feature in features_only_in_train:
    print(f'{feature}: {train[feature].dtype}') 

# also finding the number of features 
f'Number of features only in training: {len(features_only_in_train)}'

Completing data preprocessing. Will begin by merging the test and full set in order to prepare for analysis and model testing.

In [ ]:
# merging test and full set 
test_copy = test.copy()
test_merged = test_copy.merge(full[['date']+list(features_only_in_train)], how='right', on='date')  

In [ ]:
set(train.columns) - set(test_merged.columns)

In [ ]:
test_merged[['rougher.output.recovery', 'final.output.recovery']]

In [ ]:
test_merged.dtypes

In [ ]:
train.dtypes

The only feature that seems to be in the wrong dtype is the 'date' column in both the test and training sets. For now, I'll go ahead and remove that feature as well as the index column as it holds little to no weight.

In [ ]:
test_merged = test_merged.drop('date', axis=1) 
train = train.drop('date', axis=1) 

In [ ]:
# checking for duplicates
test_merged.columns.duplicated()

In [ ]:
train.columns.duplicated()

There are no duplicated columns.

In [ ]:
test_merged.duplicated().sum()

In [ ]:
train.duplicated().sum() 

In [ ]:
train = train.drop_duplicates().reset_index(drop=True) 
test_merged = test_merged.drop_duplicates().reset_index(drop=True) 

In [ ]:
train.duplicated().sum()

In [ ]:
test_merged.duplicated().sum()

No more duplicated rows.

In [ ]:
test_merged.isna().sum()

In [ ]:
train.isna().sum()

As some of the missing values are in features that may hold a lot of weight towards the target, I will be filling them in to maintain the size of the datasets as well as the integrity of the process data.

In [ ]:
test_merged = test_merged.fillna(test_merged.median())
train = train.fillna(train.median()) # filling with the median as it deals better in terms of outliers and some features may be of importance 

In [ ]:
# verifying changes
test_merged.isna().sum()

In [ ]:
train.isna().sum()

In [ ]:
# test_merged = test_merged.drop('date', axis=1)
# train = train.drop('index', axis=1) 

In [ ]:
# finding recovery calculation
c = train['rougher.output.concentrate_au']
f = train['rougher.input.feed_au']
t = train['rougher.output.tail_au']

recovery_calculation = (c*(f-t))/(f*(c-t))*100 

# calculating the MAE
valid_data = train.dropna(
    subset=[
        'rougher.output.recovery', 
        'rougher.output.concentrate_au', 
        'rougher.input.feed_au', 
        'rougher.output.tail_au'
    ]
)
valid_data['rougher.output.recovery'] = valid_data['rougher.output.recovery'].astype(np.float32) 
recovery_calculation = recovery_calculation.replace([np.inf, -np.inf], np.nan).dropna() 
valid_data = valid_data.loc[recovery_calculation.index] 
mae = mean_absolute_error(valid_data['rougher.output.recovery'], recovery_calculation) 

f'Mean Absolute Error of Recovery Calculation: {mae}' 

This is indicating a prediction difference of about 57 units from the actual recovery values. This will be interesting when looking at the MAE from the machine learning models later on.

In [ ]:
test_merged.shape

In [ ]:
# assigning the target and feature variables
target_train = train[['rougher.output.recovery', 'final.output.recovery']]
features_train = train.drop(['rougher.output.recovery', 'final.output.recovery'], axis=1) 

target_test = test_merged[['rougher.output.recovery', 'final.output.recovery']] 
features_test = test_merged.drop(['rougher.output.recovery', 'final.output.recovery'], axis=1) 

In [ ]:
test_merged['total_raw_feed'] = test_merged[['rougher.input.feed_au', 'rougher.input.feed_ag', 'rougher.input.feed_pb']].sum(axis=1)
test_merged['total_rougher_concentrate'] = test_merged[['rougher.output.concentrate_au', 'rougher.output.concentrate_ag', 'rougher.output.concentrate_pb']].sum(axis=1)
test_merged['total_final_concentrate'] = test_merged[['final.output.concentrate_au', 'final.output.concentrate_ag', 'final.output.concentrate_pb']].sum(axis=1) 

train['total_raw_feed'] = train[['rougher.input.feed_au', 'rougher.input.feed_ag', 'rougher.input.feed_pb']].sum(axis=1)
train['total_rougher_concentrate'] = train[['rougher.output.concentrate_au', 'rougher.output.concentrate_ag', 'rougher.output.concentrate_pb']].sum(axis=1)
train['total_final_concentrate'] = train[['final.output.concentrate_au', 'final.output.concentrate_ag', 'final.output.concentrate_pb']].sum(axis=1) 

# Analyze the Data

Inspecting how the concentrations of metals change depending on the purification stage. 

In [ ]:
metal_stages = [
    'rougher.output.concentrate', 
    'primary_cleaner.output.concentrate', 
    'secondary_cleaner.output.concentrate',
    'final.output.concentrate'
]
   
metals = ['au', 'ag', 'pb']

for stage in metal_stages:
    for metal in metals:
        column_name = f'{stage}_{metal}'
        if column_name in train.columns:
            print(f'{column_name}: {train[column_name].mean()}')
   

In [ ]:
metal_stages = [
    'rougher.output.concentrate',
    'primary_cleaner.output.concentrate',
    'secondary_cleaner.output.concentrate',
    'final.output.concentrate'
]

metals = ['au', 'ag', 'pb']

# plot
for metal in metals:
    plt.figure(figsize=(12, 6))
    
    for stage in metal_stages:
        column_name = f'{stage}_{metal}'
        
        if column_name in train.columns:
            plt.hist(train[column_name], bins=30, alpha=0.5, label=stage)
    
    plt.title(f'Distribution of {metal.upper()} Concentration')
    plt.xlabel('Concentration')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()

As purification progresses, the different metals (gold, silver, lead) each do different things. Just looking at the averages, the concentration of gold is the highest of the three and it continues to increase to more than double that amount by the end of the purification process. On the other hand, silver goes down in concentration and lead grows, but generally stays within the same amount. The visuals then give more insight into the fact that at the beginning of the process, there is a much higher amount of lead and silver present when compared to gold. By the end of the process, however, the presence of gold is much more apparent. The visuals also represent the information derived from the previous analysis which provided the averages of the same prompt. 

In [ ]:
# comparing feed particle size distributions in the training set and testing set
plt.figure(figsize=(12, 6))
sns.histplot(train['rougher.input.feed_size'], color='blue', label='Train', kde=True, stat="density", linewidth=0)
sns.histplot(test['rougher.input.feed_size'], color='red', label='Test', kde=True, stat="density", linewidth=0)

plt.title('Feed Particle Size Distribution')
plt.xlabel('Particle Size')
plt.ylabel('Density')
plt.legend()
plt.show()

Above is the feed particle size distributions in the training and test set. With the visual it's clear to see that the distributions are in no way significantly different, suggesting that the model evaluation will reflect accurate information and will thus be correct. 

In [ ]:
# plot histograms for each stage
stages = ['total_raw_feed', 'total_rougher_concentrate', 'total_final_concentrate']

plt.figure(figsize=(16, 8))

for i, stage in enumerate(stages, 1):
    plt.subplot(1, 3, i)
    plt.hist(train[stage], bins=30, color='skyblue', edgecolor='black')
    plt.title(stage)
    plt.xlabel('Total Concentration')
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show() 

The visual above represents the total concentration of substances at different stages of the recovery process. Here we can see that in the beginning of the process, there is a much larger total concentration compared to the next stage and the last one as well. This makes sense as the substances go through purification to have a concentration of only gold by the end of it. We can also see where there are abnormal values, outliers, where there's almost no peaks in the plots. While it makes sense that there are peaks climbing high, it doesn't make sense to have no amount of substances. With that in mind, if these values stay in the set, they may cause confusion and inaccurate findings in the future so it is best to have them removed.

In [ ]:
# removing near_zero values 
threshold = 1.0 # removes values that may be an error or irrelevant while maintaining as much integrity as possible

# remove from training set 
train = train[
    (train['total_raw_feed'] > threshold) &
    (train['total_rougher_concentrate'] > threshold) &
    (train['total_final_concentrate'] > threshold)
]

# remove from testing set
test_merged = test_merged[
    (test_merged['total_raw_feed'] > threshold) &
    (test_merged['total_rougher_concentrate'] > threshold) &
    (test_merged['total_final_concentrate'] > threshold)
] 

# Building the Model

Based on the type of data we are handling (continuous), the models that will be tested will be regression models since the target variable is not a binary outcome. 

In [ ]:
# writing the sMAPE function to use later once the values are available from the model testings
def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

In [ ]:
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test_merged) 

Training various models and determining which performs the best.

In [ ]:
common_columns = test.columns.intersection(features_train.columns)
features_train_aligned = features_train[common_columns]
features_test_aligned = features_test[common_columns] 

In [ ]:
test.shape

In [ ]:
features_train_aligned 

In [ ]:
# RandomForest model 
model_rf_multi = MultiOutputRegressor(RandomForestRegressor(random_state=123, n_estimators=10))
model_rf_multi.fit(features_train_aligned, target_train) 

In [ ]:
# cross-validation
cv_predictions = cross_val_predict(model_rf_multi, features_train_aligned, target_train, cv=5) 

In [ ]:
# calculating sMAPE 
smape_rougher = smape(target_train['rougher.output.recovery'], cv_predictions[:, 0])
smape_final = smape(target_train['final.output.recovery'], cv_predictions[:, 1]) 
smape_value = 0.25 * smape_rougher + 0.75 * smape_final
f'sMAPE: {smape_value}'

In [ ]:
# quality metrics 

# first target
mae_rf_rougher = mean_absolute_error(target_train['rougher.output.recovery'], cv_predictions[:, 0])
rmse_rf_rougher = mean_squared_error(target_train['rougher.output.recovery'], cv_predictions[:, 0], squared=False)

# second target
mae_rf_final = mean_absolute_error(target_train['final.output.recovery'], cv_predictions[:, 1])
rmse_rf_final = mean_squared_error(target_train['final.output.recovery'], cv_predictions[:, 1], squared=False)

print(f'MAE Rougher: {mae_rf_rougher}, MAE Final: {mae_rf_final}')
print(f'RMSE Rougher: {rmse_rf_rougher}, RMSE Final: {rmse_rf_final}') 

Both metrics indicate prediction accuracy, however there may be some outliers present. These in addition with the sMAPE value will be compared with another model to see which performs best with the data.

In [ ]:
# linear regression model 
model_lr_multi = MultiOutputRegressor(LinearRegression())
model_lr_multi.fit(features_train_aligned, target_train) 

In [ ]:
# cross-validation
cv_predictions_lr = cross_val_predict(model_lr_multi, features_train_aligned, target_train, cv=5)

In [ ]:
# calculating sMAPE 
smape_rougher_lr = smape(target_train['rougher.output.recovery'], cv_predictions_lr[:, 0])
smape_final_lr = smape(target_train['final.output.recovery'], cv_predictions_lr[:, 1]) 
smape_value_lr = 0.25 * smape_rougher_lr + 0.75 * smape_final_lr
f'sMAPE: {smape_value_lr}' 

In [ ]:
# quality metrics 
# first target
mae_rf_rougher_lr = mean_absolute_error(target_train['rougher.output.recovery'], cv_predictions_lr[:, 0])
rmse_rf_rougher_lr = mean_squared_error(target_train['rougher.output.recovery'], cv_predictions_lr[:, 0], squared=False)

# second target
mae_rf_final_lr = mean_absolute_error(target_train['final.output.recovery'], cv_predictions_lr[:, 1])
rmse_rf_final_lr = mean_squared_error(target_train['final.output.recovery'], cv_predictions_lr[:, 1], squared=False)

print(f'MAE Rougher: {mae_rf_rougher_lr}, MAE Final: {mae_rf_final_lr}')
print(f'RMSE Rougher: {rmse_rf_rougher_lr}, RMSE Final: {rmse_rf_final_lr}') 

Based on the values from both models, we can see that the Random Forest Regressor has lower values in all metrics tested. These indicate that this model performs better in terms of percentage error as well as accuracy. We can now move forward with final testing with this model.

In [ ]:
param_grid = {
    'estimator__n_estimators': [50, 100],
    'estimator__max_depth': [10, 20],
} 


def smape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denominator = np.abs(y_true) + np.abs(y_pred)
    denominator[denominator == 0] = 1  # Prevent division by zero
    smape_value = 100 * np.mean(2 * np.abs(y_pred - y_true) / denominator)
    return smape_value

def multi_target_smape(y_true, y_pred):
    y_true_rougher , y_pred_rougher = y_true[0], y_pred[0] 
    y_true_final, y_pred_final = y_true[1], y_pred[1] 
    smape_value_lr = 0.25 * smape(y_true_rougher, y_pred_rougher) + 0.75 * smape(y_true_final, y_pred_final) 
    return smape_value_lr 

smape_scorer = make_scorer(multi_target_smape, greater_is_better=False) 

In [ ]:
target_train_array = target_train.values if hasattr(target_train, 'values') else target_train

In [ ]:
# sample 1000 rows randomly
sample_size = 1000

# randomly select
sample_indices = target_train.sample(n=sample_size, random_state=42).index

# create samples 
sample_train = features_train.loc[sample_indices]
sample_target = target_train.loc[sample_indices]

# convert to arrays
sample_train_array = sample_train.values
sample_target_array = sample_target.values

In [ ]:
grid_search = GridSearchCV(MultiOutputRegressor(RandomForestRegressor(random_state=123)), param_grid, scoring=smape_scorer, cv=3, n_jobs=-1, verbose=2)
   
grid_search.fit(sample_train_array, sample_target_array)

In [ ]:
best_params = grid_search.best_params_
print("Best Parameters:", best_params)
print("Best sMAPE Score:", grid_search.best_score_)

The model is performing at a high rate. 

In [ ]:
# final testing
final_model = MultiOutputRegressor(RandomForestRegressor(random_state=123, n_estimators=best_params['estimator__n_estimators'], max_depth=best_params['estimator__max_depth']))
final_model.fit(features_train_aligned, target_train)

In [ ]:
# predictions
final_predictions = final_model.predict(features_test_aligned)

In [ ]:
def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred))) 

In [ ]:
final_smape = multi_target_smape(target_test.values, final_predictions)
print(f'Final sMAPE: {final_smape}')

In [ ]:
# final metrics 
mae_rougher_test = mean_absolute_error(target_test['rougher.output.recovery'], final_predictions[:, 0])
rmse_rougher_test = mean_squared_error(target_test['rougher.output.recovery'], final_predictions[:, 0], squared=False)

mae_final_test = mean_absolute_error(target_test['final.output.recovery'], final_predictions[:, 1])
rmse_final_test = mean_squared_error(target_test['final.output.recovery'], final_predictions[:, 1], squared=False)

print(f'MAE Rougher Test: {mae_rougher_test}, MAE Final Test: {mae_final_test}')
print(f'RMSE Rougher Test: {rmse_rougher_test}, RMSE Final Test: {rmse_final_test}') 

During final testing, I evaluated the same metrics as before when finding a model to train. Thankfully, these metrics are the best ones so far, meaning the model is performing at a high level. Accuracy and predictions are at its highest so far. Let's do some checks to see more information on how the model is performing.

In [ ]:
# calculate means for constant predictions
mean_rougher = target_train['rougher.output.recovery'].mean()
mean_final = target_train['final.output.recovery'].mean()

# create constant predictions based on means
constant_predictions = np.full((len(target_test), 2), [mean_rougher, mean_final])

# calculate sMAPE for constant model
smape_rougher_constant = smape(target_test['rougher.output.recovery'], constant_predictions[:, 0])
smape_final_constant = smape(target_test['final.output.recovery'], constant_predictions[:, 1])
smape_constant_value = 0.25 * smape_rougher_constant + 0.75 * smape_final_constant

# print constant model sMAPE
print(f'sMAPE Constant Model: {smape_constant_value}') 

In [ ]:
# looking at feature importance
importance = final_model.estimators_[0].feature_importances_
plt.barh(range(len(importance)), importance)
plt.yticks(range(len(importance)), features_train_aligned.columns)
plt.xlabel('Feature Importance')
plt.show()

This visual is important as the goal of this project is to eliminate wasteful parameters. Here, we can see the features that the model relies on more than others. While these features make sense as to why they're given a bit more weight, it's interesting to see how little others are contributing, especially since the model is predicting so well already. These features may contribute to overfitting and if removed can improve the model even more.

In [ ]:
residuals = target_test['rougher.output.recovery'] - final_predictions[:, 0]
sns.histplot(residuals, bins=20, kde=True)
plt.title('Residuals for Rougher')
plt.show()

The above visual shows the distribution of prediction for the final model. The peak being at zero suggests that most predictions are indeed accurate. The symmetry of the plot also suggests that the model is fairly predicting both positive and negtive errors. Then looking at the sMAPE value from the sanity check, we can see that it is significantly larger than the sMAPE value from the final model. So in terms of capturing complexity and variability, the final model captures the data's complexity well. Keeping all of this in mind, the final model is currently predicting with high accuracy and low bias.

# Conclusion

The aim of this project was to reduce wasteful parameters, optimize production, and find a model that would best predict the amount of gold recovered from gold ore. Loading the data went without problem as there were no major issues to address. Immediately, I took a look at the recovery calculation and found that it suggested extreme accuracy to the real values. I then moved on to examining the differences between the test and training set (which was helpful information when performing model testing). Here I found a difference in the amount of columns. While at this point, I also looked into duplicates, missing values, and dtypes. I merged the full set with the test set to gain targets while also removing unnecessary features. Once the data was complete and ready for analysis, I looked at the concentration of metals at various stages of purification. I found that gold increased with each subsequent stage while silver went down, and lead essentially stayed around the same amount. Keeping in mind the purpose of the process, this information made sense. I also examined the feed particle distribution between the training and test set in preparation for model training. Thankfully, the values were matching up, giving security that the values received from the models would be accurate. I also looked at the total concentrations of substances at different stages. This showed us that in the beginning of the recovery process is when there's the most amount of substances present. Then the amount decreases as the stages progress, but the concentration also increases. Thinking back to the last visual, this makes sense. The gold increases while the others decrease. I moved on to conclude this project with model training. Based on the type of data we are working with (continuous), only regression models would be of use as they would not be predicting targets that are binary. I began with a RandomForestRegressor and then moved on to train a LinearRegression model as well. I examined quality metrics consisting of the sMAPE value, MAE, and RMSE. With the RandomForestRegressor model, all of the metrics tested came in at better quality with the RandomForestRegressor. Then with final testing, we achieved the lowest scores yet, suggesting that the model is performing at a high quality and is extremely accurate. The feature importances and residuals then showed which features contribute most to this success and which may be contributing to overfitting. A great majority of the features towards the end of the recovery process are contributing very little (if at all) towards the model's total performance. The sanity check showed that the final model is performing at a high rate, handling variability and patterns well. With the information about the various metals, the various stages, and the performance of the model,  we will most definitely be able to improve the way gold is recovered from gold ore from a business perspective. 